In [1]:
from google.colab import files
uploaded = files.upload()

Saving apache_logs.txt to apache_logs.txt


###1. Data Preparation
Phase the Data

In [2]:
from datetime import datetime
import pytz
#%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-agent}i\"
def parse_str(x):
    """
    Returns the string delimited by two characters.
    Example:
        `>>> parse_str('[my string]')`
        `'my string'`
    """
    if x is None:
        return '-'
    return x[1:-1]

In [3]:
parse_str('[my string]')

'my string'

In [4]:
def parse_int(x):
    """Returns parsed string if no error occured during parse else returns 0"""
    if x is None:
        return 0
    try:
        y = int(x)
        return y
    except ValueError:
        return 0

In [5]:
test_ = "234"
parse_int(test_)

234

In [6]:
def parse_datetime(x):
    '''
    Parses datetime with timezone formatted as:
        `[day/month/year:hour:minute:second zone]`

    Example:
        `>>> parse_datetime('13/Nov/2015:11:45:42 +0000')`
        `datetime.datetime(2015, 11, 3, 11, 45, 4, tzinfo=<UTC>)`

    Due to problems parsing the timezone (`%z`) with `datetime.strptime`, the
    timezone will be obtained using the `pytz` library.
    '''
    try:
        dt = datetime.strptime(x[1:-7], '%d/%b/%Y:%H:%M:%S')
        dt_tz = int(x[-6:-3])*60+int(x[-3:-1])
        return dt.replace(tzinfo=pytz.FixedOffset(dt_tz))
    except ValueError:
        return '-'


In [7]:
parse_datetime('13/Nov/2015:11:45:42 +0000')

datetime.datetime(2015, 11, 3, 11, 45, 4, tzinfo=<UTC>)

## Read data from apache log file

In [8]:
import re
import pandas as pd
conv = {'time': parse_datetime,
        'request': parse_str,
        'status': parse_int,
        'size': parse_int,
        'referer': parse_str,
        'user_agent': parse_str}

data = pd.read_csv('./apache_logs.txt',
                    header=None,
                    sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
                    engine='python',
                    na_values='-',
                    usecols=[0, 3, 4, 5, 6, 7, 8],
                    #names=['ip', 'time', 'request', 'status', 'size', 'referer', 'user_agent'],
                    names=['Client_IP_address', 'Time_of_activity','Requested_Page', 'Status_Code', 'Size_of_Page', 'Referer_page','Client_Device'],
                    converters=conv,on_bad_lines='skip')
data.head(4)

,Client_IP_address,Time_of_activity,Requested_Page,Status_Code,Size_of_Page,Referer_page,Client_Device
0,83.149.9.216,[17/May/2015:10:05:03 +0000],"""GET /presentations/logstash-monitorama-2013/i...",200,203023.0,"""http://semicomplete.com/presentations/logstas...","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1..."
1,83.149.9.216,[17/May/2015:10:05:43 +0000],"""GET /presentations/logstash-monitorama-2013/i...",200,171717.0,"""http://semicomplete.com/presentations/logstas...","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1..."
2,83.149.9.216,[17/May/2015:10:05:47 +0000],"""GET /presentations/logstash-monitorama-2013/p...",200,26185.0,"""http://semicomplete.com/presentations/logstas...","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1..."
3,83.149.9.216,[17/May/2015:10:05:12 +0000],"""GET /presentations/logstash-monitorama-2013/p...",200,7697.0,"""http://semicomplete.com/presentations/logstas...","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1..."


In [9]:
# counting unique values from IP and Devices
IP=set(data['Client_IP_address'])
print(len(IP))
devices=set(data['Client_Device'])
print(len(devices))

1754
559


## 2. Identifying 10 Frequent Visitors

This cell is iterating over the pandas DataFrame named data, which presumably contains web visitation or event data with at least two columns: 'Client_IP_address' and 'Client_Device'. The purpose of the code is to count the number of occurrences (visits or events) for each unique combination of client IP address and client device.

###Breakdown :

- It initializes an empty dictionary D.
- It then iterates through each row of the DataFrame data using iterrows(), which provides the index (i) and the data (row) for each row.
- For each row, it checks if the combination of 'Client_IP_address' and 'Client_Device' exists as a tuple key in the dictionary D.
- If the key exists, it increments the value associated with that key by 1.

This increment signifies another occurrence of that specific IP-device combination.
If the key does not exist, it adds the tuple of 'Client_IP_address' and 'Client_Device' as a new key to the dictionary D and sets its value to 1, indicating the first occurrence of this unique combination.

In [10]:
D={}
for i,row in data.iterrows():
    if(tuple((row['Client_IP_address'],row['Client_Device'])) in D):
        D[tuple((row['Client_IP_address'],row['Client_Device']))]+=1

    else:
        D[tuple((row['Client_IP_address'],row['Client_Device']))]=1

This cell below sorts the dictionary D based on the values (the counts of unique combinations of IP addresses and client devices) in descending order and then selects the top 10 most frequent combinations.

In [13]:
freq = sorted(D, key = D.get, reverse = True)


top_10 = freq[:10]
for i,j in enumerate(top_10, start=1):
    print(i,"",j)

1  ('46.105.14.53', '"UniversalFeedParser/4.2-pre-314-svn +http://feedparser.org/"')
2  ('130.237.218.86', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.91 Safari/537.36"')
3  ('75.97.9.59', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.107 Safari/537.36"')
4  ('66.249.73.135', '"Mozilla/5.0 (iPhone; CPU iPhone OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/6.0 Mobile/10A5376e Safari/8536.25 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"')
5  ('66.249.73.135', '"Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"')
6  ('50.16.19.13', '"Tiny Tiny RSS/1.11 (http://tt-rss.org/)"')
7  ('68.180.224.225', '"Mozilla/5.0 (compatible; Yahoo! Slurp; http://help.yahoo.com/help/us/ysearch/slurp)"')
8  ('208.115.111.72', '"Mozilla/5.0 (compatible; Ezooms/1.0; help@moz.com)"')
9  ('198.46.149.143', '"Tiny Tiny RSS/1.11 (http://tt-rss.or

Here are the results of the top 10 users who visited frequently.

## Sessions and page views per session

The cell below creates three lists (time, page, and al) based on the first 500 unique combinations of IP addresses and client devices stored in the dictionary D.

In [12]:
time=[]
page=[]
al=[]
for i in list(D)[:500]:
    for j,row in data.iterrows():
        if(i==tuple((row['Client_IP_address'],row['Client_Device']))):
            time.append(row['Time_of_activity'])
            page.append(row['Requested_Page'])
            al.append(i)


In [ ]:
al[:5]

[('83.149.9.216',
  '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"'),
 ('83.149.9.216',
  '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"'),
 ('83.149.9.216',
  '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"'),
 ('83.149.9.216',
  '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"'),
 ('83.149.9.216',
  '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"')]

In [ ]:
time[:5]

['[17/May/2015:10:05:03 +0000]',
 '[17/May/2015:10:05:43 +0000]',
 '[17/May/2015:10:05:47 +0000]',
 '[17/May/2015:10:05:12 +0000]',
 '[17/May/2015:10:05:07 +0000]']

In [ ]:
page[:5]

['"GET /presentations/logstash-monitorama-2013/images/kibana-search.png HTTP/1.1"',
 '"GET /presentations/logstash-monitorama-2013/images/kibana-dashboard3.png HTTP/1.1"',
 '"GET /presentations/logstash-monitorama-2013/plugin/highlight/highlight.js HTTP/1.1"',
 '"GET /presentations/logstash-monitorama-2013/plugin/zoom-js/zoom.js HTTP/1.1"',
 '"GET /presentations/logstash-monitorama-2013/plugin/notes/notes.js HTTP/1.1"']

In [ ]:
df=pd.DataFrame({'Time':time,'Pages':page,'Client':al})
df.head(40)

,Time,Pages,Client
0,[17/May/2015:10:05:03 +0000],"""GET /presentations/logstash-monitorama-2013/i...","(83.149.9.216, ""Mozilla/5.0 (Macintosh; Intel ..."
1,[17/May/2015:10:05:43 +0000],"""GET /presentations/logstash-monitorama-2013/i...","(83.149.9.216, ""Mozilla/5.0 (Macintosh; Intel ..."
2,[17/May/2015:10:05:47 +0000],"""GET /presentations/logstash-monitorama-2013/p...","(83.149.9.216, ""Mozilla/5.0 (Macintosh; Intel ..."
3,[17/May/2015:10:05:12 +0000],"""GET /presentations/logstash-monitorama-2013/p...","(83.149.9.216, ""Mozilla/5.0 (Macintosh; Intel ..."
4,[17/May/2015:10:05:07 +0000],"""GET /presentations/logstash-monitorama-2013/p...","(83.149.9.216, ""Mozilla/5.0 (Macintosh; Intel ..."
5,[17/May/2015:10:05:34 +0000],"""GET /presentations/logstash-monitorama-2013/i...","(83.149.9.216, ""Mozilla/5.0 (Macintosh; Intel ..."
6,[17/May/2015:10:05:57 +0000],"""GET /presentations/logstash-monitorama-2013/c...","(83.149.9.216, ""Mozilla/5.0 (Macintosh; Intel ..."
7,[17/May/2015:10:05:50 +0000],"""GET /presentations/logstash-monitorama-2013/c...","(83.149.9.216, ""Mozilla/5.0 (Macintosh; Intel ..."
8,[17/May/2015:10:05:24 +0000],"""GET /presentations/logstash-monitorama-2013/i...","(83.149.9.216, ""Mozilla/5.0 (Macintosh; Intel ..."
9,[17/May/2015:10:05:50 +0000],"""GET /presentations/logstash-monitorama-2013/i...","(83.149.9.216, ""Mozilla/5.0 (Macintosh; Intel ..."


From the above data frame we can see:

(83.149.9.216, "Mozilla/5.0 (Macintosh; Intel): 22 pageviews


(24.236.252.67, "Mozilla/5.0 (X11) : 1 pageview


(93.114.45.13, "Mozilla/5.0 (X11; Linux x86_64): 6 pageviews


(66.249.73.135, "Mozilla/5.0 (iPhone; CPU): 10 page views

## Top Five most referrer websites



This cell processes a list of URLs from a 'Referer_page' column in a pandas DataFrame named data, aiming to identify and display the top 5 most frequently occurring domains (excluding any potential parsing errors represented by ':///'), including their schemes (e.g., http or https)

In [ ]:
referers = data['Referer_page'].dropna(how='all')
from urllib.parse import urlparse
ref=[]
for i in referers:
    parsed_uri = urlparse(i.replace('"', ''))
    result = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    ref.append(result)
word_counter = {}
for url in ref:
    if url in word_counter:
        word_counter[url] += 1
    else:
        word_counter[url] = 1
popular_words = sorted(word_counter, key = word_counter.get, reverse = True)
top_5 = popular_words[:6]
for i in top_5:
    if(i==':///'''):
        top_5.remove(i)
for i,j in enumerate(top_5,start=1):
    print(i,"",j)

1  http://www.semicomplete.com/
2  http://semicomplete.com/
3  http://www.google.com/
4  https://www.google.com/
5  http://stackoverflow.com/


## Ten most frequent patterns

The code below is organizing the pages visited by different clients during their sessions.


Here's a breakdown of what each part of the code does:

1. `Pages_visited_per_session = list(df['Pages'])`: Extracts the column 'Pages' from the DataFrame `df` and converts it into a list named `Pages_visited_per_session`. This list likely contains the pages visited by clients during their sessions.

2. `Pages_accessed = {}`: Initializes an empty dictionary to store pages accessed by each client.

3. Iterating over rows and populating `Pages_accessed`:
   - The code iterates over each row in the DataFrame `df`.
   - For each row, it checks if the client's ID (`row['Client']`) is already in the `Pages_accessed` dictionary.
   - If the client ID is present, it appends the page visited (`row['Pages']`) to the list of pages accessed by that client.
   - If the client ID is not present, it creates a new key-value pair where the key is the client ID and the value is a list containing the page visited.

4. Printing pages accessed by each client:
   - After populating `Pages_accessed`, the code iterates over the dictionary.
   - For each client, it prints a message indicating the client's ID and the pages accessed by that client.

5. `all_pages=[]`: Initializes an empty list to store all pages accessed by all clients.

6. Collecting all pages accessed:
   - The code iterates over each client in `Pages_accessed`.
   - For each client, it appends the list of pages accessed by that client to the `all_pages` list.

Then returns us a dictionary `Pages_accessed` where each key represents a client ID, and the corresponding value is a list of pages accessed by that client. Additionally, you'll have a list `all_pages` containing all pages accessed by all clients.

In [ ]:
Pages_visited_per_session=list(df['Pages'])
Pages_accessed={}
for i,row in df.iterrows():
    if(row['Client'] in Pages_accessed):
        Pages_accessed[row['Client']].append(row['Pages'])
    else:
        Pages_accessed[row['Client']]=[row['Pages']]
for i in Pages_accessed:
    print('\n Client {} accessed these pages \n {} \n'.format(i,Pages_accessed[i]))
all_pages=[]
for i in Pages_accessed:
    all_pages.append(Pages_accessed[i])


 Client ('83.149.9.216', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"') accessed these pages 
 ['"GET /presentations/logstash-monitorama-2013/images/kibana-search.png HTTP/1.1"', '"GET /presentations/logstash-monitorama-2013/images/kibana-dashboard3.png HTTP/1.1"', '"GET /presentations/logstash-monitorama-2013/plugin/highlight/highlight.js HTTP/1.1"', '"GET /presentations/logstash-monitorama-2013/plugin/zoom-js/zoom.js HTTP/1.1"', '"GET /presentations/logstash-monitorama-2013/plugin/notes/notes.js HTTP/1.1"', '"GET /presentations/logstash-monitorama-2013/images/sad-medic.png HTTP/1.1"', '"GET /presentations/logstash-monitorama-2013/css/fonts/Roboto-Bold.ttf HTTP/1.1"', '"GET /presentations/logstash-monitorama-2013/css/fonts/Roboto-Regular.ttf HTTP/1.1"', '"GET /presentations/logstash-monitorama-2013/images/frontend-response-codes.png HTTP/1.1"', '"GET /presentations/logstash-monitorama-2013/images/kibana-das

In [ ]:
!pip install apyori

### Performing association rule mining with the Apriori algorithm

In [ ]:
from apyori import apriori
association_rules = apriori(all_pages, min_support=0.1, min_confidence=0.5, min_lift=1, min_length=7)
association_results = list(association_rules)
support=[]
items=[]
rhs=[]
lhs=[]
con=[]
lift=[]
for i in association_results:
    support.append(i.support)
    items.append(i.items)
    rhs.append(i.ordered_statistics[0][1])
    lhs.append(i.ordered_statistics[0][0])
    con.append(i.ordered_statistics[0][2])
    lift.append(i.ordered_statistics[0][3])
df=pd.DataFrame({'Items':items,'Antecedent':lhs,'Precedent':rhs,'Support':support,'Confidence':con,'Lift':lift})

In [ ]:
df.head()

,Items,Antecedent,Precedent,Support,Confidence,Lift
0,"(""GET /images/jordan-80.png HTTP/1.1"", ""GET /f...","(""GET /favicon.ico HTTP/1.1"")","(""GET /images/jordan-80.png HTTP/1.1"")",0.180,0.580645,2.592166
1,"(""GET /images/web/2009/banner.png HTTP/1.1"", ""...","(""GET /favicon.ico HTTP/1.1"")","(""GET /images/web/2009/banner.png HTTP/1.1"")",0.180,0.580645,2.639296
2,"(""GET /reset.css HTTP/1.1"", ""GET /favicon.ico ...","(""GET /favicon.ico HTTP/1.1"")","(""GET /reset.css HTTP/1.1"")",0.180,0.580645,2.524544
3,"(""GET /style2.css HTTP/1.1"", ""GET /favicon.ico...","(""GET /favicon.ico HTTP/1.1"")","(""GET /style2.css HTTP/1.1"")",0.180,0.580645,2.502781
4,"(""GET /images/jordan-80.png HTTP/1.1"", ""GET /i...","(""GET /images/jordan-80.png HTTP/1.1"")","(""GET /images/web/2009/banner.png HTTP/1.1"")",0.216,0.964286,4.383117


The dataframe we have above contains information about the association rules mined from the pages accessed by clients, including the support (frequency), confidence, and lift for each rule. This information can be further analyzed and interpreted to gain insights into the browsing patterns and behaviors of clients on the website.

## Extracting meaningful association rules


This loop iterates over each row in the filtered DataFrame s_p, where rules with support greater than 0.20 are considered. It prints the antecedent, precedent, support, confidence, and lift for each association rule meeting the support threshold

In [ ]:
s_p=df[df['Support']>0.20].sort_values('Support',ascending=False)
for i,items in df.iterrows():
    # first index of the inner list
    # Contains base item and add item
#     pair = item[0]
#     items = [x for x in pair]
#     print(item[0])
    print("Rule: " + str(items['Antecedent']) + " -> " + str(items['Precedent']))
    #second index of the inner list
    print("Support: " + str(items['Support']))
    #third index of the list located at 0th
    #of the third index of the inner list
    print("Confidence: " + str(items['Confidence']))
    print("Lift: " + str(items['Lift']))
    print("=====================================")

Rule: frozenset({'"GET /favicon.ico HTTP/1.1"'}) -> frozenset({'"GET /images/jordan-80.png HTTP/1.1"'})
Support: 0.18
Confidence: 0.5806451612903225
Lift: 2.592165898617511
Rule: frozenset({'"GET /favicon.ico HTTP/1.1"'}) -> frozenset({'"GET /images/web/2009/banner.png HTTP/1.1"'})
Support: 0.18
Confidence: 0.5806451612903225
Lift: 2.6392961876832843
Rule: frozenset({'"GET /favicon.ico HTTP/1.1"'}) -> frozenset({'"GET /reset.css HTTP/1.1"'})
Support: 0.18
Confidence: 0.5806451612903225
Lift: 2.5245441795231414
Rule: frozenset({'"GET /favicon.ico HTTP/1.1"'}) -> frozenset({'"GET /style2.css HTTP/1.1"'})
Support: 0.18
Confidence: 0.5806451612903225
Lift: 2.5027808676307
Rule: frozenset({'"GET /images/jordan-80.png HTTP/1.1"'}) -> frozenset({'"GET /images/web/2009/banner.png HTTP/1.1"'})
Support: 0.216
Confidence: 0.9642857142857143
Lift: 4.383116883116883
Rule: frozenset({'"GET /images/jordan-80.png HTTP/1.1"'}) -> frozenset({'"GET /reset.css HTTP/1.1"'})
Support: 0.214
Confidence: 0.955

In [ ]:
s_p

,Items,Antecedent,Precedent,Support,Confidence,Lift
9,"(""GET /reset.css HTTP/1.1"", ""GET /style2.css H...","(""GET /reset.css HTTP/1.1"")","(""GET /style2.css HTTP/1.1"")",0.226,0.982609,4.235382
4,"(""GET /images/jordan-80.png HTTP/1.1"", ""GET /i...","(""GET /images/jordan-80.png HTTP/1.1"")","(""GET /images/web/2009/banner.png HTTP/1.1"")",0.216,0.964286,4.383117
5,"(""GET /reset.css HTTP/1.1"", ""GET /images/jorda...","(""GET /images/jordan-80.png HTTP/1.1"")","(""GET /reset.css HTTP/1.1"")",0.214,0.955357,4.153727
6,"(""GET /style2.css HTTP/1.1"", ""GET /images/jord...","(""GET /images/jordan-80.png HTTP/1.1"")","(""GET /style2.css HTTP/1.1"")",0.214,0.955357,4.117919
18,"(""GET /reset.css HTTP/1.1"", ""GET /images/jorda...","(""GET /images/jordan-80.png HTTP/1.1"")","(""GET /reset.css HTTP/1.1"", ""GET /style2.css H...",0.214,0.955357,4.227244
7,"(""GET /reset.css HTTP/1.1"", ""GET /images/web/2...","(""GET /images/web/2009/banner.png HTTP/1.1"")","(""GET /reset.css HTTP/1.1"")",0.212,0.963636,4.189723
8,"(""GET /style2.css HTTP/1.1"", ""GET /images/web/...","(""GET /images/web/2009/banner.png HTTP/1.1"")","(""GET /style2.css HTTP/1.1"")",0.212,0.963636,4.153605
16,"(""GET /reset.css HTTP/1.1"", ""GET /images/jorda...","(""GET /images/jordan-80.png HTTP/1.1"")","(""GET /reset.css HTTP/1.1"", ""GET /images/web/2...",0.212,0.946429,4.464286
17,"(""GET /style2.css HTTP/1.1"", ""GET /images/jord...","(""GET /images/jordan-80.png HTTP/1.1"")","(""GET /style2.css HTTP/1.1"", ""GET /images/web/...",0.212,0.946429,4.464286
19,"(""GET /reset.css HTTP/1.1"", ""GET /images/web/2...","(""GET /images/web/2009/banner.png HTTP/1.1"")","(""GET /reset.css HTTP/1.1"", ""GET /style2.css H...",0.212,0.963636,4.263878


## Install Generalized Sequential Patterns Algorithm

Sequential Pattern Mining Using GSP Algorithm

In [14]:
!pip install gsppy

In [ ]:
import argparse
import logging
import random
from gsppy.gsp import GSP
logging.basicConfig(level=logging.DEBUG)

The GSP algorithm will analyze the sequences of pages accessed by clients and identify frequent sequential patterns that occur with a support greater than or equal to the specified threshold (0.1 in this case). The result will contain these sequential patterns along with their support values

In [ ]:
transactions=all_pages
result = GSP(transactions).search(0.1)

DEBUG:root:
        Run 1
        There are 1217 candidates.
        The candidates have been filtered down to 5.

DEBUG:root:
        Run 2
        There are 25 candidates.
        The candidates have been filtered down to 4.

DEBUG:root:
        Run 3
        There are 125 candidates.
        The candidates have been filtered down to 2.

DEBUG:root:
        Run 4
        There are 256 candidates.
        The candidates have been filtered down to 1.

DEBUG:root:
        Run 5
        There are 1024 candidates.
        The candidates have been filtered down to 0.



In [ ]:
result

[{('"GET /favicon.ico HTTP/1.1"',): 155,
  ('"GET /reset.css HTTP/1.1"',): 115,
  ('"GET /style2.css HTTP/1.1"',): 116,
  ('"GET /images/jordan-80.png HTTP/1.1"',): 112,
  ('"GET /images/web/2009/banner.png HTTP/1.1"',): 110},
 {('"GET /reset.css HTTP/1.1"', '"GET /style2.css HTTP/1.1"'): 98,
  ('"GET /images/web/2009/banner.png HTTP/1.1"',
   '"GET /favicon.ico HTTP/1.1"'): 54,
  ('"GET /style2.css HTTP/1.1"', '"GET /images/jordan-80.png HTTP/1.1"'): 69,
  ('"GET /images/jordan-80.png HTTP/1.1"',
   '"GET /images/web/2009/banner.png HTTP/1.1"'): 94},
 {('"GET /reset.css HTTP/1.1"',
   '"GET /style2.css HTTP/1.1"',
   '"GET /images/jordan-80.png HTTP/1.1"'): 65,
  ('"GET /style2.css HTTP/1.1"',
   '"GET /images/jordan-80.png HTTP/1.1"',
   '"GET /images/web/2009/banner.png HTTP/1.1"'): 62},
 {('"GET /reset.css HTTP/1.1"',
   '"GET /style2.css HTTP/1.1"',
   '"GET /images/jordan-80.png HTTP/1.1"',
   '"GET /images/web/2009/banner.png HTTP/1.1"'): 59}]